In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock

from case_study_trains import *
import case_study_trains

m = wt.watertap_setup(dynamic = False)

# wt.case_study_trains.train = {"case_study": "KBHDP",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "Irwin",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "EMWD",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

wt.case_study_trains.train = {"case_study": "Santa_Barbara",
                             "reference": "NAWI",
                             "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "Carlsbad",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "Ashkelon",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}


# TODO LATER: how to make this sync with info in train input data. We might not need to do that.
#But, if the source water type is different to what is in the train (pfd dictionary), 
#then we should updat the node name. If more than two sources - what to do? Needs to be
#based on pfd node!?

# wt.case_study_trains.source_water = {"case_study": "KBHDP", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "KBHDP_Brackish_Ave"}

# wt.case_study_trains.source_water = {"case_study": "Irwin", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "Brackish"}

# wt.case_study_trains.source_water = {"case_study": "EMWD", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "EMWD_CA_Brackish"}

wt.case_study_trains.source_water = {"case_study": "Santa_Barbara", 
                             "reference": "NAWI",
                             "scenario": "Baseline",
                             "water_type": "Seawater"}

# wt.case_study_trains.source_water = {"case_study": "Carlsbad", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "Seawater"}

# wt.case_study_trains.source_water = {"case_study": "Ashkelon", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "Seawater"}

m = wt.case_study_trains.get_case_study(m = m) # flow is set as case study flow unless defined.

Santa_Barbara
sw_onshore_intake
ferric_chloride_addition
chlorination
static_mixer
treated_storage
tri_media_filtration
backwash_solids_handling
anti_scalant_addition
cartridge_filtration
ro_deep
ro_deep_2
ro_deep_3
holding_tank
uv_aop
co2_addition
lime_softening
treated_storage_b
municipal_drinking
surface_discharge


In [2]:
wt.display.show_train2(model_name=m)

In [3]:
wt.run_water_tap(m = m, solver_results = True, print_model_results = True)

degrees_of_freedom: 25
Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3600
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      843

Total number of variables............................:     1147
                     variables with only lower bounds:       38
                variables with lower and upper bounds:        3
                     variables with only upp

In [ ]:
m.fs.holding_tank.pprint()

In [5]:
train = wt.case_study_trains.train

py_var = [
    "fixed_cap_inv",
    "total_cap_investment",
    "cat_and_chem_cost",
    "electricity_cost",
    "total_fixed_op_cost"]

excel_var = ['Fixed Capital Investment (FCI)',
            'Total Capital Investment (TCI)',
            'Catalysts and Chemicals',
            'Electricity',
            'Total Fixed Operating Costs']

case_study = train['case_study']
print(f'\n######### Case study = {case_study} #########\n\n\n')

cap_inv_tot = []

big_dict = {}

for b_unit in m.fs.component_objects(Block, descend_into=True):
    unit = str(b_unit)[3:]
    if hasattr(b_unit, 'costing'):
        print(f'____________{unit}____________')
        py_vals =  []
        excel_vals = []
        big_dict[unit] = {'python': {}, 'excel': {}}
#         print(f'\nflow in --> {round(value(b_unit.inlet.flow_vol[0.0]), 4) * 3600}')
#         print(f'flow out --> {round(value(b_unit.outlet.flow_vol[0.0]), 4) * 3600}')
#         print(f'waste --> {round(value(b_unit.waste.flow_vol[0.0]), 4) * 3600}\n')
        print(f'\nflow in --> {round(value(b_unit.inlet.flow_vol[0.0]), 4)}')
        print(f'flow out --> {round(value(b_unit.outlet.flow_vol[0.0]), 4)}')
        print(f'waste --> {round(value(b_unit.waste.flow_vol[0.0]), 4)}\n')
        for py, ex in dict(zip(py_var, excel_var)).items():
            print(f'\n#### {py}')
            
            try:
                num = value(getattr(b_unit.costing, py))
                py_vals.append(num)
                print(f'Python --> {round(num, 5)}')
            except ZeroDivisionError:
                print(f'Python --> {0} - ERROR')
                py_vals.append(0)
            py_dict = dict(zip(py_var, py_vals))
            df = pd.read_csv("data/case_study_results.csv")
            df = df[df.case_study == train['case_study']]
            df = df[df.scenario == train['scenario']]
            df = df[df.unit_process == unit]
            num = df[df.Variable == ex].value.max()
            print(f'Excel --> {round(num, 5)}\n\n')
            excel_vals.append(num)
            excel_dict = dict(zip(py_var, excel_vals))
        big_dict[unit]['python'] = py_dict
        big_dict[unit]['excel'] = excel_dict
        
else:
    pass 

df = pd.DataFrame.from_dict({(i,j): big_dict[i][j] 
                           for i in big_dict.keys() 
                           for j in big_dict[i].keys()},
                           orient='index')




######### Case study = Santa_Barbara #########



____________sw_onshore_intake____________

flow in --> 0.3092
flow out --> 0.3092
waste --> -0.0


#### fixed_cap_inv
Python --> 1.9102
Excel --> 0.72167



#### total_cap_investment
Python --> 2.00857
Excel --> 0.75884



#### cat_and_chem_cost
Python --> 0.0
Excel --> 0.0



#### electricity_cost
Python --> 0.13389
Excel --> 0.04459



#### total_fixed_op_cost
Python --> 0.02665
Excel --> 0.01185


____________ferric_chloride_addition____________

flow in --> 0.3092
flow out --> 0.3092
waste --> -0.0


#### fixed_cap_inv
Python --> 0.46084
Excel --> 1.10361



#### total_cap_investment
Python --> 0.48458
Excel --> 1.16044



#### cat_and_chem_cost
Python --> 0.24898
Excel --> 0.0572



#### electricity_cost
Python --> 0.0
Excel --> 0.0



#### total_fixed_op_cost
Python --> 0.00645
Excel --> 0.01446


____________chlorination____________

flow in --> 0.3092
flow out --> 0.3092
waste --> -0.0


#### fixed_cap_inv
Python --> 4.95175
Ex

'2'

'ro_deep'

In [ ]:
# df_final = pd.DataFrame()
# for case_name in ["Ashkelon", "Carlsbad", "Tampa_Bay"]: #, "Santa_Barbra"]: #, "Ashkelon"]:

#     m = wt.watertap_setup(dynamic = False)

#     wt.case_study_trains.train = {"case_study": case_name,
#                                  "reference": "NAWI",
#                                  "scenario": "Baseline"}

#     # TODO LATER: how to make this sync with info in train input data. We might not need to do that.
#     #But, if the source water type is different to what is in the train (pfd dictionary), 
#     #then we should updat the node name. If more than two sources - what to do? Needs to be
#     #based on pfd node!?

#     wt.case_study_trains.source_water = {"case_study": case_name, 
#                                  "reference": "NAWI",
#                                  "scenario": "Baseline",
#                                  "water_type": "Seawater"}
    
#     m = wt.case_study_trains.get_case_study(m = m) # flow is set as case study flow unless defined.
    
      
#     wt.run_water_tap(m = m, solver_results = False, print_model_results = False)
#     print(value(m.fs.costing.LCOW))
#     df = get_results_table()
#     df["case_study"] = case_name
#     df_final = pd.concat([df_final,df])
    

In [ ]:
def get_results_table():

    train = wt.case_study_trains.train

    py_var = [
        "fixed_cap_inv",
        "total_cap_investment",
        "cat_and_chem_cost",
        "electricity_cost",
        "total_fixed_op_cost"]

    excel_var = ['Fixed Capital Investment (FCI)',
                'Total Capital Investment (TCI)',
                'Catalysts and Chemicals',
                'Electricity',
                'Total Fixed Operating Costs']
    big_dict = {}
    # for letter in ['A', 'B', 'C']:
    for b_unit in m.fs.component_objects(Block, descend_into=True):
        unit = str(b_unit)[3:]
        if hasattr(b_unit, 'costing'):
            #print(f'____________{unit}____________')
            py_vals =  []
            excel_vals = []
            big_dict[unit] = {'python': {}, 'excel': {}}
            for py, ex in dict(zip(py_var, excel_var)).items():
                #print(f'\n#### {py}')
                try:
                    num = value(getattr(b_unit.costing, py))
                    py_vals.append(num)
                    #print(f'Python --> {num}')
                except ZeroDivisionError:
                    #print(f'Python --> {0} - ERROR')
                    py_vals.append(0)
                py_dict = dict(zip(py_var, py_vals))
                df = pd.read_csv("data/case_study_results.csv")
                df = df[df.case_study == train['case_study']]
                df = df[df.scenario == train['scenario']]
                df = df[df.unit_process == unit]
                num = df[df.Variable == ex].value.max()
                #print(f'Excel --> {num}\n\n')
                excel_vals.append(num)
                excel_dict = dict(zip(py_var, excel_vals))
            big_dict[unit]['python'] = py_dict
            big_dict[unit]['excel'] = excel_dict
    else:
        pass 



    df = pd.DataFrame.from_dict({(i,j): big_dict[i][j] 
                               for i in big_dict.keys() 
                               for j in big_dict[i].keys()},
                           orient='index')
    
    return df

In [ ]:
get_results_table()

In [ ]:
df = pd.read_csv("data/water_recovery.csv")
case_study_name = case_study_trains.train["case_study"]
unit_process_type = 'uv_aop'
unit_process_name = unit_process_type
import generate_constituent_list

if case_study_name in df[df.unit_process == unit_process_type].case_study:
    if "calculated" not in df[
        ((df.unit_process == unit_process_type) & (df.case_study == case_study_name))].recovery.max():
        flow_recovery_factor = float(
            df[((df.unit_process == unit_process_type) & (df.case_study == case_study_name))].recovery)
        getattr(m.fs, unit_process_name).water_recovery.fix(flow_recovery_factor)
else:
    if "calculated" not in df[
        ((df.unit_process == unit_process_type) & (df.case_study == "Default"))].recovery.max():
        flow_recovery_factor = float(
            df[((df.unit_process == unit_process_type) & (df.case_study == "Default"))].recovery)
        getattr(m.fs, unit_process_name).water_recovery.fix(flow_recovery_factor)

# Get constituent list and removal rates for this unit process
train_constituent_removal_factors = generate_constituent_list.get_removal_factors(unit_process_type)

for constituent_name in getattr(m.fs, unit_process_name).config.property_package.component_list:

    if constituent_name in train_constituent_removal_factors.keys():
        getattr(m.fs, unit_process_name).removal_fraction[:, constituent_name].fix(
            train_constituent_removal_factors[constituent_name])
    else:
        getattr(m.fs, unit_process_name).removal_fraction[:, constituent_name].fix(0)
# Also set pressure drops - for now I will set these to zero
getattr(m.fs, unit_process_name).deltaP_outlet.fix(1e-4)
getattr(m.fs, unit_process_name).deltaP_waste.fix(1e-4)

In [ ]:
df = get_results_table()

In [ ]:
df.to_csv("tmp/test_compare.csv")

In [20]:
train = wt.case_study_trains.train
source_water = case_study_trains.source_water
import importfile
df = pd.read_csv("data/constituent_removal.csv")
df = df[df.reference == train["reference"]]
df = df[df.case_study == train["case_study"]]
df = df[df.scenario == train["scenario"]]

df2 = importfile.feedwater(
        input_file="data/case_study_water_sources.csv",
        reference = source_water["reference"], 
        water_type = source_water["water_type"], 
        case_study = source_water["case_study"],
        scenario = source_water["scenario"])
    
    # gets list of consituents in inlet water
list2 = df.index

list1 = df[df.value >=0].constituent.unique()

final_list = [x for x in list1 if x in list2]

In [ ]:
Excel_variable
Python_variable
Unit

Fixed Capital Investment (FCI)
fixed_cap_inv
$MM

Catalysts and Chemicals
cat_and_chem_cost
$MM/yr

Electricity
electricity_cost
$MM/yr

Other Operating
other_var_cost
$MM/yr

Total Fixed Operating 
total_fixed_op_cost
$MM/yr

Levelized Cost
LCOW
$/m3






In [ ]:
cost_range_list = []; #results will be inputted in this array
#up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.

#for value_change in pct_to_target1: # cycles through each value from MC range
for value_change in [0.4, 0.8]: #, 0.9]:

    # create and build model
    m = wt.watertap_setup(dynamic = False)
    m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

    m.fs.tri_media_filtration.water_recovery.fix(value_change)

    # set variable to MC value
    wt.run_water_tap(m)
    results_table = get_results_table(m, unit_process_name)
    cost_range_list.append(results_table.total_up_cost.sum())


In [ ]:
cost_range_list

In [ ]:
#### DO NOT USE THE BELOW ####

In [ ]:
import time
from multiprocessing import Pool
import multiprocessing

mu = 0.6
sigma = .1
num_reps = 50

input_list = np.random.normal(mu,sigma, size = num_reps) #, sigma, num_reps).round(4)

count, bins, ignored = plt.hist(input_list, 25, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
          linewidth=2, color='r')
plt.show()

### INPUT TO MODEL LIST: ### CAN BE AUTOMATED FOR USER TO LABEL THE VARIABLE. TOOD ###
no_of_proc = 4
list_final = []
for i in range(no_of_proc):
    part2 = len(input_list) / no_of_proc
    i2 = ((i+1)*part2)
    list1 = input_list[int(i*part2):int(i2)]
    list_final.append(list1)
    
    
def monte_run(list_final):
    print('goes in')

    up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.
    cost_range_list = []; #results will be inputted in this array

    #for value_change in pct_to_target1: # cycles through each value from MC range
    for value_change in list_final:

        # create and build model
        m = wt.watertap_setup(dynamic = False)
        m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

        getattr(m.fs, up_name).water_recovery.fix(value_change)

        # set variable to MC value
        result = wt.run_water_tap(m)
        results_table = get_results_table(m, unit_process_names)
        cost_range_list.append(results_table.total_up_cost.sum())


    return cost_range_list

startTime = time.time()

pool=Pool()
dfs = pool.map(monte_run, list_final) #SomeClass().preprocess_data()

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
####TO DO LOAD AND SAVE!!

In [ ]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [ ]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [ ]:
# wt.display.show_train(TT)